## Get Next Experimental Trials

In [1]:
import os
import numpy as np
import pandas as pd
from ax.service.ax_client import AxClient, ObjectiveProperties

# hide future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
SEED = 3 # students set this to their group number

filename = 'running_trials_trial2.csv'

### Set up the experiment and specify parameter types and value ranges

In [2]:
ax_client = AxClient(random_seed=SEED)

# define the experiment and objectives
ax_client.create_experiment(
    parameters=[
        {"name": "PINK", "type": "range", "bounds": [0, 20], "value_type": "int"},
        {"name": "GREEN", "type": "range", "bounds": [0, 20], "value_type": "int"},
        {"name": "BLUE", "type": "range", "bounds": [0, 20], "value_type": "int"},
        {"name": "PURPLE", "type": "range", "bounds": [0, 20], "value_type": "int"},
        {"name": "SOLID_LIQUID_RATIO", "type": "range", "bounds": [0.5, 3.0], "value_type": "float", "digits": 1},
    ],
    objectives={
        "COLOR_ERR": ObjectiveProperties(minimize=True, threshold=1e-3),
        "STRAIN_ERR": ObjectiveProperties(minimize=True, threshold=1e-3),
    },
    choose_generation_strategy_kwargs={"num_initialization_trials": 0},
)

[INFO 04-23 10:12:18] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[WARNING 04-23 10:12:18] ax.service.ax_client: Random seed set to 3. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 04-23 10:12:18] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='PINK', parameter_type=INT, range=[0, 20]), RangeParameter(name='GREEN', parameter_type=INT, range=[0, 20]), RangeParameter(name='BLUE', parameter_type=INT, range=[0, 20]), RangeParameter(name='PURPLE', parameter_type=INT, range=[0, 20]), RangeParameter(name='SOLID_LIQUID_RA

### Load in the previous experimental results, train a model, and predict the next set of trials

In [74]:
# instantiate a running_experiment file if it doesn't exist
if not os.path.exists(os.path.join('data', 'running_trials.csv')):
    try:
        os.system("copy data\\sobol_trials.csv data\\running_trials.csv")
    except:
        os.system("cp data/sobol_trials.csv data/running_trials.csv")        

# read in the previous trials data and correct the data types
running_trials = pd.read_csv(os.path.join('data', 'running_trials.csv')).astype({'PINK': int, 'GREEN': int, 'BLUE': int, 'PURPLE': int})

# separate parameters from results
X_train = running_trials.iloc[:,:5].to_dict(orient="records")
y_train = running_trials.iloc[:,5:]#.to_dict(orient="records")
y_train = [{k:v for (k,v) in zip(["COLOR_ERR", "STRAIN_ERR"], [(row[0], row[1]), (row[2], row[3])])} for row in y_train.values]

# add the previous trials to the experiment
for i in range(len(running_trials)):
    parameterization = X_train[i]
    ax_client.attach_trial(parameterization)
    ax_client.complete_trial(trial_index=i, raw_data=y_train[i])

# generate new trials
batch_size = 5
parameterizations, optimization_complete = ax_client.get_next_trials(batch_size)
trials_df = ax_client.get_trials_data_frame()
display(trials_df)

[INFO 04-22 19:45:00] ax.core.experiment: Attached custom parameterizations [{'PINK': 1, 'GREEN': 1, 'BLUE': 1, 'PURPLE': 1, 'SOLID_LIQUID_RATIO': 1.3}] as trial 0.
[INFO 04-22 19:45:00] ax.service.ax_client: Completed trial 0 with data: {'COLOR_ERR': (10.0, 1.0), 'STRAIN_ERR': (22.97, 10.99)}.
[INFO 04-22 19:45:00] ax.core.experiment: Attached custom parameterizations [{'PINK': 1, 'GREEN': 1, 'BLUE': 1, 'PURPLE': 1, 'SOLID_LIQUID_RATIO': 1.5}] as trial 1.
[INFO 04-22 19:45:00] ax.service.ax_client: Completed trial 1 with data: {'COLOR_ERR': (10.0, 1.0), 'STRAIN_ERR': (19.67, 5.57)}.
[INFO 04-22 19:45:00] ax.core.experiment: Attached custom parameterizations [{'PINK': 1, 'GREEN': 1, 'BLUE': 1, 'PURPLE': 1, 'SOLID_LIQUID_RATIO': 2.5}] as trial 2.
[INFO 04-22 19:45:00] ax.service.ax_client: Completed trial 2 with data: {'COLOR_ERR': (10.0, 1.0), 'STRAIN_ERR': (4.73, 2.78)}.
[INFO 04-22 19:45:00] ax.core.experiment: Attached custom parameterizations [{'PINK': 1, 'GREEN': 1, 'BLUE': 1, 'PU

[INFO 04-22 19:45:01] ax.core.experiment: Attached custom parameterizations [{'PINK': 1, 'GREEN': 1, 'BLUE': 1, 'PURPLE': 1, 'SOLID_LIQUID_RATIO': 2.0}] as trial 7.
[INFO 04-22 19:45:01] ax.service.ax_client: Completed trial 7 with data: {'COLOR_ERR': (10.0, 1.0), 'STRAIN_ERR': (2.5, 0.5)}.
[INFO 04-22 19:45:01] ax.modelbridge.transforms.standardize_y: Outcome COLOR_ERR is constant, within tolerance.
c:\Users\bmill\miniconda3\envs\MatInformatics\lib\site-packages\botorch\models\utils\assorted.py:202: InputDataWarning: Input data is not standardized (mean = tensor([0.0000e+00, 5.5511e-17], dtype=torch.float64), std = tensor([0.0000, 1.0000], dtype=torch.float64)). Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)
[INFO 04-22 19:45:04] ax.service.ax_client: Generated new trial 8 with parameters {'PINK': 2, 'GREEN': 2, 'BLUE': 3, 'PURPLE': 7, 'SOLID_LIQUID_RATIO': 2.2} using model BoTorch.
[INFO 04-22 19:45:04] ax.modelbridge.transfor

,trial_index,arm_name,trial_status,generation_method,is_feasible,COLOR_ERR,STRAIN_ERR,PINK,GREEN,BLUE,PURPLE,SOLID_LIQUID_RATIO
0,0,0_0,COMPLETED,Manual,False,10.0,22.97,1,1,1,1,1.30
1,1,1_0,COMPLETED,Manual,False,10.0,19.67,1,1,1,1,1.50
2,2,2_0,COMPLETED,Manual,False,10.0,4.73,1,1,1,1,2.50
3,3,3_0,COMPLETED,Manual,False,10.0,100.00,1,1,1,1,0.70
4,4,4_0,COMPLETED,Manual,False,10.0,100.00,1,1,1,1,0.80
5,5,1_0,COMPLETED,Manual,False,10.0,19.44,1,1,1,1,1.50
6,6,6_0,COMPLETED,Manual,False,10.0,1.00,1,1,1,1,2.25
7,7,7_0,COMPLETED,Manual,False,10.0,2.50,1,1,1,1,2.00
8,8,8_0,RUNNING,BoTorch,NaN,NaN,NaN,2,2,3,7,2.20
9,9,9_0,RUNNING,BoTorch,NaN,NaN,NaN,3,3,4,6,2.20


## Next Trials To Run

In [75]:
display(trials_df.iloc[-3:, :])

,trial_index,arm_name,trial_status,generation_method,is_feasible,COLOR_ERR,STRAIN_ERR,PINK,GREEN,BLUE,PURPLE,SOLID_LIQUID_RATIO
8,8,8_0,RUNNING,BoTorch,NaN,NaN,NaN,2,2,3,7,2.2
9,9,9_0,RUNNING,BoTorch,NaN,NaN,NaN,3,3,4,6,2.2
10,10,10_0,RUNNING,BoTorch,NaN,NaN,NaN,3,1,2,7,2.2
